In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import bnk

# Structure

## Atom Structure

In [ ]:
atom = bnk.HSpace(2, name='atom')

atom_excite = atom.eigenstate(1) @ atom.eigenstate(0).ct
atom_relax = atom_excite.ct


plt.subplot(1, 2, 1)
plt.matshow(atom_excite.flattened_values, 0)
plt.title("atom excite")

plt.subplot(1, 2, 2)
plt.matshow(atom_relax.flattened_values, 0)
plt.title("atom relax")

plt.show()

## Field Structure

In [ ]:
field = bnk.HSpace(4, name='field')

field_excite = 0
for i in range(1, field.n):
    field_excite_i = field.eigenstate(i) @ field.eigenstate(i - 1).ct
    field_excite += np.sqrt(i) * field_excite_i
field_relax = field_excite.ct


plt.subplot(1, 2, 1)
plt.matshow(field_excite.flattened_values, 0)
plt.title("field excite")

plt.subplot(1, 2, 2)
plt.matshow(field_relax.flattened_values, 0)
plt.title("field relax")

plt.show()

# Evolution

## Hamiltonian

In [ ]:
hb = 1
w = 1
g = 0.3

interaction = g * (0
    + atom_relax @ field_excite
    + atom_excite @ field_relax
    + atom_excite @ field_excite
    + atom_relax @ field_relax
)

energy = hb * w * (0
    + atom_excite @ atom_relax
    + field_excite @ field_relax
)

hamiltonian = energy + interaction


plt.matshow(hamiltonian.flattened_values)
plt.title("hamiltonian")
plt.show()

# Iteration

## Initial $\psi$

In [ ]:
psi0 = atom.eigenstate(0) @ field.eigenstate(1)

print('psi0')
print(psi0.values)

## Configurations

In [ ]:
dt = 1e-5
mt = 50

dlt = mt/100

## Iterations

In [ ]:
%%time

_, _, logs_t, logs_v = bnk.evolve_schrodinger_with_logs(
    0.0, psi0, hamiltonian, hb,
    mt, dt, dlt)

In [ ]:
def plot_logs(logs_t, logs_v):
    logs_v = [v.values for v in logs_v]
    logs_p = np.conj(logs_v) * logs_v
    logs_angle = np.angle(logs_v) / np.pi

    plt.figure()
    plt.plot(logs_t, logs_p[:,1,0], label='atom')
    plt.plot(logs_t, logs_p[:,0,1], label='field')
    plt.legend()
    plt.xlabel("time")
    plt.ylabel("prob")
    plt.show()

    plt.figure()
    plt.plot(logs_t, logs_angle[:,1,0], label='atom')
    plt.plot(logs_t, logs_angle[:,0,1], label='field')
    plt.legend()
    plt.xlabel("time")
    plt.ylabel("angle")
    plt.show()

plot_logs(logs_t,logs_v)

## Reduction

In [ ]:
reduced_space = bnk.ReducedHSpace.from_seed(
    [psi0],[hamiltonian],'reduced_system')

print([dim.name for dim in reduced_space.org_eigenstates[0].dims])
for i, org_eigenstate in enumerate(reduced_space.org_eigenstates):
    code = list(np.transpose(np.where(org_eigenstate.values))[0])
    print(i, code)

In [ ]:
reduced_psi0 = reduced_space.reduce(psi0)
reduced_hamiltonian = reduced_space.reduce(hamiltonian)

print('reduced psi0')
print(reduced_psi0.values)

plt.matshow(reduced_hamiltonian.flattened_values)
plt.title("reduced hamiltonian")
plt.show()

## Iterations (reduced)

In [ ]:
%%time

_, _, logs_t, logs_v = bnk.evolve_schrodinger_with_logs(
    0.0, reduced_psi0, reduced_hamiltonian, hb,
    mt, dt, dlt)

In [ ]:
logs_v = [reduced_space.inflate(v) for v in logs_v]
plot_logs(logs_t,logs_v)